Installing package into 'C:/Users/user/AppData/Local/R/win-library/4.3'
(as 'lib' is unspecified)



Warning message in install.packages("http(s)://DSS_HOST:DSS_PORT/public/packages/dataiku_current.tar.gz", :
"installation of package 'http(s)://DSS_HOST:DSS_PORT/public/packages/dataiku_current.tar.gz' had non-zero exit status"


In [3]:
# Load necessary libraries
library('ggplot2')
library('dplyr')
library('tidyr')
library('hrbrthemes')
library('dygraphs')
library('xts')          # To make the convertion data-frame / xts format
library('tidyverse')
library('lubridate')



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Loading required package: zoo


Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric



######################### Warning from 'xts' package ##########################
#                                                                             #
# The dplyr lag() function breaks how base R's lag() function is supposed to  #
# work, which breaks lag(my_xts). Calls to lag(my_xts) that you type or       #
# source() into this session won't work correctly.                            #
#                                                                             #
# Use stats::lag() to make sure you're not using dplyr::lag(), or you can add #
# conflictRules('dplyr', exclude = 'lag') to your .Rprofile to stop           #
# dplyr from 

In [4]:
artists_data0 <- read.csv("songs_normalize.csv")

In [15]:
# Preprocessing: Standardize gender capitalization
artists_data <- artists_data0 %>%
  filter(year != 1998 ) %>%  # Remove entries from the year 1998
  filter(year != 1999 ) %>%  # Remove entries from the year 1999
  filter(year != 2020 ) %>%  # Remove entries from the year 2020
  mutate(gender = case_when(
    tolower(gender) == "male" ~ "Male",
    tolower(gender) == "female" ~ "Female",
    TRUE ~ as.character(gender) # Keeps original value if not "male" or "female"
  ))

genre_counts <- artists_data %>%
  group_by(gender, genre) %>%
  summarise(Artist_Count = n(), .groups = 'drop')  # Replace n() with count(distinct(artist)) if artists are not unique

top_genres <- genre_counts %>%
  arrange(gender, desc(Artist_Count)) %>%
  group_by(gender) %>%
  slice_head(n = 5)  # This selects the top 5 rows for each gender group

print(top_genres)

# Create separate barplots for Male and Female
# Assuming top_genres is already calculated as per your snippet
# Create separate barplots for Male and Female, with ordered bars and specific colors
ggplot(top_genres, aes(x = reorder_within(genre, Artist_Count, gender), y = Artist_Count, fill = gender)) +
  geom_bar(stat = "identity") +  # Use geom_bar and set stat to "identity" for pre-summarized counts
  facet_wrap(~gender, scales = "free_x") +  # Separate plots for each gender
  scale_x_reordered() +  # Helper function from 'ggplot2' to handle reordering within facets
  labs(title = "Top 5 Genres by Gender",
       x = "Genre",
       y = "Artist Count") +
  theme_minimal() +  # Minimal theme for a cleaner look
  scale_fill_manual(values = c("Male" = "blue", "Female" = "red")) +  # Specify colors for each gender
  coord_flip() +  # Flip the coordinates to make the barplots horizontal
  theme(axis.text.x = element_text(angle = 0, hjust = 1),  # Adjust text alignment and angle
        legend.position = "none")  # Hide the legend if not needed

# A tibble: 10 × 3
# Groups:   gender [2]
   gender genre                 Artist_Count
   <chr>  <chr>                        <int>
 1 Female pop                            206
 2 Female pop, R&B                       133
 3 Female hip hop, pop, R&B               80
 4 Female pop, Dance/Electronic           80
 5 Female hip hop, pop                    15
 6 Male   hip hop, pop                   260
 7 Male   pop                            210
 8 Male   hip hop, pop, R&B              160
 9 Male   pop, Dance/Electronic          139
10 Male   hip hop                        121


ERROR: Error in scale_x_reordered(): could not find function "scale_x_reordered"


# A tibble: 40 × 3
    year gender Count
   <int> <chr>  <int>
 1  2000 Female    27
 2  2000 Male      47
 3  2001 Female    55
 4  2001 Male      53
 5  2002 Female    35
 6  2002 Male      55
 7  2003 Female    40
 8  2003 Male      57
 9  2004 Female    29
10  2004 Male      67
# ℹ 30 more rows
